In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import qiskit as qk
qk.IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
class Model:
    def __init__(self,n_work,n_simulation,n_ancilla,Emax,*argv):
        self.w = n_work
        self.s = n_simulation
        self.N = n_work + n_simulation + n_ancilla
        self.qb = qk.QuantumRegister(self.N)
        self.cb = qk.ClassicalRegister(self.N)
        self.qc = qk.QuantumCircuit(self.qb,self.cb)
        self.Emax = Emax
        self.args = argv
        self.plotX = np.zeros(1)
        self.plotY = np.zeros(1)
        self.result = {}
    
    def toPhase(self,bitstr):
        phase = 0
        for i,c in enumerate(bitstr):
            phase += (2**(-i-1))*int(c)
        return phase
        
    def measure(self,t):
        self.qc.measure(self.qb,self.cb)
        job = qk.execute(self.qc, backend = qk.Aer.get_backend('qasm_simulator'), shots=1024)
        #qk.tools.monitor.job_monitor(job)
        result = job.result().get_counts(self.qc)
        x = [] # phase
        y = [] # hits
        for key,val in result.items():
            eigenstate = key[:self.N-self.w]
            phi = key[self.N-self.w:]
            phi = phi[::-1]
            x.append(self.Emax - 2*np.pi*self.toPhase(phi)/t)
            y.append(val)
        x = np.array(x)
        y = np.array(y)
        idx = np.argsort(x)
        x = x[idx]
        y = y[idx]
        ## Check if same phase measured for different eigenstates
        x_ = []
        y_ = []
        for i,xi in enumerate(x):
            if i > 0:
                if xi == x_[-1]:
                    y_[-1] += y[i]
                else:
                    x_.append(xi)
                    y_.append(y[i])

            else:
                x_.append(xi)
                y_.append(y[i])
        self.plotX = x_
        self.plotY = y_
        self.result = result
        return self

In [3]:
def simple(Qcirc,w,ctrl,dt):
    qc = Qcirc[0]
    qb = Qcirc[1]
    cb = Qcirc[2]
    
    qc.cx(qb[w],qb[-1])
    qc.cx(qb[w],qb[-1])
    qc.crz(2*dt,qb[ctrl],qb[w+1])
    
    qc.u1(2*dt,qb[ctrl])

    return [qc,qb,cb]

In [4]:
class Pairing(Model):
    def __init__(self,*args):
        super().__init__(*args)
        self.delta = self.args[0]
        self.g = self.args[1]
        self.ansatz()
    
    def __call__(self,control_qubit,dt):
        n_work = self.w
        n_simulation = self.s
        n_qubits = self.N

        qz = self.qc
        qb = self.qb
        
        s_state = 0

        for q_state in range(0,n_simulation):
            if q_state % 2 == 0:
                s_state += 1
            qz.crz(dt*delta*(s_state - 1),qb[control_qubit],qb[q_state+n_work])

            qz.cu1(-dt*delta*(1/8)*(n_simulation-2)*n_simulation,qb[control_qubit],qb[n_work])
            qz.x(qb[n_work])
            qz.cu1(-dt*delta*(1/8)*(n_simulation-2)*n_simulation,qb[control_qubit],qb[n_work])
            qz.x(qb[n_work])

            qz.cu1(Emax*dt,qb[control_qubit],qb[n_work])
            qz.x(qb[n_work])
            qz.cu1(Emax*dt,qb[control_qubit],qb[n_work])
            qz.x(qb[n_work])


            for p in range(1,n_simulation,2):
                for q in range(p,n_simulation,2):
                    if p == q:
                        theta = -2*(1/8)*g*dt
                        qz.cu1(-theta/2,qb[control_qubit],qb[n_work])
                        qz.x(qb[n_work])
                        qz.cu1(-theta/2,qb[control_qubit],qb[n_work])
                        qz.x(qb[n_work])

                        qz.crz(theta,qb[control_qubit],qb[p-1+n_work])
                        qz.crz(theta,qb[control_qubit],qb[p+n_work])

                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.crz(theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                    else:
                        theta = -2*(1/16)*g*dt
                        #FIRST TERM:
                        qz.h(qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.crz(theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.h(qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        ############
                        #SECOND TERM:
                        qz.h(qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.rz(np.pi/2,qb[q-1+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.rz(np.pi/2,qb[q+n_work])
                        qz.h(qb[q+n_work])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.crz(-theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.h(qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.rz(-np.pi/2,qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        qz.rz(-np.pi/2,qb[q+n_work])
                        ###########
                        #THIRD TERM:
                        qz.h(qb[p-1+n_work])
                        qz.rz(np.pi/2,qb[p+n_work])
                        qz.h(qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.rz(np.pi/2,qb[q+n_work])
                        qz.h(qb[q+n_work])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.crz(theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.h(qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.rz(-np.pi/2,qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        qz.rz(-np.pi/2,qb[q+n_work])
                        ###########
                        #FOURTH TERM
                        qz.h(qb[p-1+n_work])
                        qz.rz(np.pi/2,qb[p+n_work])
                        qz.h(qb[p+n_work])
                        qz.rz(np.pi/2,qb[q-1+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.crz(theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.h(qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.rz(-np.pi/2,qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.rz(-np.pi/2,qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        ###########
                        #FIFTH TERM
                        qz.rz(np.pi/2,qb[p-1+n_work])
                        qz.h(qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.rz(np.pi/2,qb[q+n_work])
                        qz.h(qb[q+n_work])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.crz(theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.h(qb[p-1+n_work])
                        qz.rz(-np.pi/2,qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        qz.rz(-np.pi/2,qb[q+n_work])
                        ##########
                        #SIXTH TERM:
                        qz.rz(np.pi/2,qb[p-1+n_work])
                        qz.h(qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.rz(np.pi/2,qb[q-1+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.crz(theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.h(qb[p-1+n_work])
                        qz.rz(-np.pi/2,qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.rz(-np.pi/2,qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        #######################
                        #SEVENTH TERM
                        qz.rz(np.pi/2,qb[p-1+n_work])
                        qz.h(qb[p-1+n_work])
                        qz.rz(np.pi/2,qb[p+n_work])
                        qz.h(qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.crz(-theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.h(qb[p-1+n_work])
                        qz.rz(-np.pi/2,qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.rz(-np.pi/2,qb[p+n_work])
                        qz.h(qb[q-1+n_work])

                        qz.h(qb[q+n_work])
                        ##############
                        #EIGTH TERM:
                        qz.rz(np.pi/2,qb[p-1+n_work])
                        qz.h(qb[p-1+n_work])
                        qz.rz(np.pi/2,qb[p+n_work])
                        qz.h(qb[p+n_work])
                        qz.rz(np.pi/2,qb[q-1+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.rz(np.pi/2,qb[q+n_work])
                        qz.h(qb[q+n_work])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.crz(theta,qb[control_qubit],qb[n_qubits-1])
                        qz.cx(qb[p-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[p+n_work],qb[n_qubits-1])
                        qz.cx(qb[q-1+n_work],qb[n_qubits-1])
                        qz.cx(qb[q+n_work],qb[n_qubits-1])
                        qz.h(qb[p-1+n_work])
                        qz.rz(-np.pi/2,qb[p-1+n_work])
                        qz.h(qb[p+n_work])
                        qz.rz(-np.pi/2,qb[p+n_work])
                        qz.h(qb[q-1+n_work])
                        qz.rz(-np.pi/2,qb[q-1+n_work])
                        qz.h(qb[q+n_work])
                        qz.rz(-np.pi/2,qb[q+n_work])
        self.qc = qz
        self.qb = qb
        return self
    
    def ansatz(self):
        for i in range(0,self.s,2):
            self.qc.h(self.qb[self.w+i])
            self.qc.cx(self.qb[self.w+i],self.qb[self.w+i+1])
        return None

In [5]:
class Heisenberg(Model):
    def __init__(self,*args):
        super().__init__(*args)
        self.h0 = self.args[0]
        self.ansatz()
    
    def __call__(self,ctrl,dt):
        qc = self.qc
        qb = self.qb
        w = self.w
        h0 = self.h0
        
        ### ONEBODY ###
        #qc.crz(h0*dt,qb[ctrl],qb[w+0])
        #qc.crz(h0*dt,qb[ctrl],qb[w+1])

        ### TWOBODY ###
        ##### X #####
        qc.cx(qb[w+1],qb[w+0])
        qc.h(qb[w+1])
        qc.crz(2*dt,qb[ctrl],qb[w+1])
        qc.h(qb[w+1])
        qc.cx(qb[w+1],qb[w+0])
        ##### Y #####
        qc.rz(np.pi/2,qb[w+0])
        qc.rz(np.pi/2,qb[w+1])
        qc.cx(qb[w+1],qb[w+0])
        qc.h(qb[w+1])
        qc.crz(2*dt,qb[ctrl],qb[w+1])
        qc.h(qb[w+1])
        qc.cx(qb[w+1],qb[w+0])
        qc.rz(-np.pi/2,qb[w+0])
        qc.rz(-np.pi/2,qb[w+1])
        ##### Z #####
        qc.cx(qb[w+1],qb[w+0])
        qc.crz(2*dt,qb[ctrl],qb[w+0])
        qc.cx(qb[w+1],qb[w+0])
        
        qc.u1(Emax*dt,qb[ctrl])

        self.qc = qc
        self.qb = qb
        
        return self
    
    def ansatz(self):
        for i in range(self.s):
            self.qc.h(self.qb[self.w+i])
        return None

In [6]:
def InvFourierTransform(model):
    qc = model.qc
    qb = model.qb
    w = model.w
    
    for i in range(int(w/2)):
        qc.swap(qb[i],qb[w-i-1])   
    for i in range(w):
        for j in range(i):
            qc.cu1(-2*np.pi/2**(i-j+1),qb[j],qb[i])
        qc.h(qb[i])

    model.qc = qc
    model.qb = qb
    return model

In [7]:
def PhaseEstimation(model,t=0.5,dt=0.005):

    s = model.s
    w = model.w
    
    # Initialize / Create superposition
    for i in range(w):
        model.qc.h(model.qb[i])
    
    # model ansatz in model.__init__
        
    # Apply controlled-U operations
    for i in range(model.w):
        for n in range(int(t/dt)):
            model = model(w-1-i,(2**i)*dt)
    
    # Inverse Quantum Fourier Transform
    model = InvFourierTransform(model)
        
    # Measurement
    model = model.measure(t)
    
    return model

In [ ]:
dt = 0.005
steps = 100
t = steps*dt

h0 = 1
delta = 1
g = 1
Emax = 2
sim = 4
work = 6
ancilla = 1
#model = Pairing(work,sim,ancilla,Emax,delta,g)
model = Pairing(6,4,1,2,1,1)
#model = Heisenberg(8,2,0,4,1)

model = PhaseEstimation(model,t=t,dt=dt)

In [ ]:
x = model.plotX
y = model.plotY
plt.plot(x,y)
plt.show()

In [ ]:
res = model.result
measurements = []
for key,value in res.items():
    key_ = key[sim:]
    eigenstate = key[1:(sim)]
    eigenstate = eigenstate[::-1]
    decimal = 0
    for i,bit in enumerate(key_):
        decimal += int(bit)*2**(-i-1)
    if value != 0:
        measurements.append(np.array([eigenstate, model.Emax-decimal*2*np.pi/t, value]))

measurements = np.array(measurements)
x = measurements[:,1]
y = measurements[:,2]
idx = np.argsort(x)
x = x[idx]
y = y[idx]
eigdict = {}
for xi in x:
    eigdict[xi] = 0
for xi, yi in zip(x,y):
    eigdict[xi] += int(yi)

x = np.array(list(eigdict.keys())).astype(np.float)
idx = np.argsort(x)
y = np.array(list(eigdict.values())).astype(np.int)
x = x[idx]
y = y[idx]
plt.plot(x,y)
plt.show()

In [ ]:
print(model.N-model.w)

In [ ]:
print(model.result)

In [ ]:
bit = '011111000'
print(len(bit[2:]))